In [5]:
import pandas as pd

# given a trades_df and quotes_df, return a df with all trades with interpolated bids and asks
def get_taq(trades_df, quotes_df):
  # drop rows with missing data
  trades_df = trades_df[(trades_df["PRICE"] != "") & (trades_df["SYM_ROOT"] == "SPY")]
  quotes_df = quotes_df[(quotes_df["ASKSIZ"] != "") & (quotes_df["SYM_ROOT"] == "SPY")]

  # create trades dataframe
  trades_df['DATETIME'] = pd.to_datetime(trades_df['DATE'] + ' ' + trades_df['TIME_M'], format='%Y-%m-%d %H:%M:%S.%f')
  trades_df = trades_df.rename(columns={'SYM_ROOT': 'TICKER'})
  trades_df = trades_df[['DATETIME', 'TICKER', 'SIZE', 'PRICE']]
  trades_df.dropna(inplace=True)

  # create quotes dataframe
  quotes_df['DATETIME'] = pd.to_datetime(quotes_df['DATE'] + ' ' + quotes_df['TIME_M'], format='%Y-%m-%d %H:%M:%S.%f')
  quotes_df['DATE'] = quotes_df['DATETIME'].dt.date
  quotes_df['DATE'] = pd.to_datetime(quotes_df['DATE'])
  quotes_df = quotes_df.rename(columns={'SYM_ROOT': 'TICKER'})

  # Group by 'TICKER', 'DATE', and 'EX' and count the entries
  grouped_df = quotes_df.groupby(['TICKER', 'DATE', 'EX']).size().reset_index(name='EntryCount')
  # Find the top 5 exchanges for each group
  top_exchanges = grouped_df.groupby(['TICKER', 'DATE']).apply(lambda x: x.nlargest(5, 'EntryCount')).reset_index(drop=True)
  # Merge on 'TICKER', 'DATE', and 'EX'
  quotes_df = pd.merge(quotes_df, top_exchanges[['TICKER', 'DATE', 'EX']], on=['TICKER', 'DATE', 'EX'], how='right')

  # get bids dataframe
  bids_df = quotes_df[['DATETIME', 'TICKER', 'BID', 'BIDSIZ']]
  bids_df.dropna(inplace=True)
  bids_df = bids_df[bids_df['BID'] != 0]
  bids_df = bids_df[bids_df['BIDSIZ'] != 0]
  bids_df = bids_df.groupby(['DATETIME', 'TICKER'])['BID'].max().reset_index()
  bids_df = bids_df[['DATETIME', 'TICKER', 'BID']]

  # get asks dataframe
  asks_df = quotes_df[['DATETIME', 'TICKER', 'ASK', 'ASKSIZ']]
  asks_df.dropna(inplace=True)
  asks_df = asks_df[asks_df['ASK'] != 0]
  asks_df = asks_df[asks_df['ASKSIZ'] != 0]
  asks_df = asks_df.groupby(['DATETIME', 'TICKER'])['ASK'].min().reset_index()
  asks_df = asks_df[['DATETIME', 'TICKER', 'ASK']]

  # merge trades with bids
  merged_df = pd.merge(trades_df, bids_df, on=['DATETIME', 'TICKER'], how='outer')
  merged_df.sort_values(by=['DATETIME'], inplace=True)
  groups = merged_df.groupby(['TICKER', merged_df['DATETIME'].dt.date])
  def interpolate_bid(group):
      group['BID'] = group['BID'].interpolate()
      return group
  merged_df = groups.apply(interpolate_bid).reset_index(drop=True)
  merged_df.dropna(inplace=True)

  # merge trades with asks
  merged_df = pd.merge(merged_df, asks_df, on=['DATETIME', 'TICKER'], how='outer')
  merged_df.sort_values(by=['DATETIME'], inplace=True)
  groups = merged_df.groupby(['TICKER', merged_df['DATETIME'].dt.date])
  def interpolate_ask(group):
      group['ASK'] = group['ASK'].interpolate()
      return group
  merged_df = groups.apply(interpolate_ask).reset_index(drop=True)
  merged_df.dropna(inplace=True)

  # drop trades/quotes that are outliers
  z_score_threshold = 3
  # merged_df['PRICE_Z_SCORE'] = (merged_df['PRICE'] - merged_df['PRICE'].mean()) / merged_df['PRICE'].std()
  merged_df['ASK_Z_SCORE'] = (merged_df['BID'] - merged_df['BID'].mean()) / merged_df['BID'].std()
  merged_df['BID_Z_SCORE'] = (merged_df['ASK'] - merged_df['ASK'].mean()) / merged_df['ASK'].std()
  merged_df = merged_df[(abs(merged_df['ASK_Z_SCORE']) <= z_score_threshold) &
                        (abs(merged_df['BID_Z_SCORE']) <= z_score_threshold)]
  merged_df['SPREAD'] = merged_df['ASK'] - merged_df['BID'] 
  merged_df = merged_df[(merged_df['PRICE'] <= merged_df['ASK'] + merged_df['SPREAD']) &
                        (merged_df['PRICE'] >= merged_df['BID'] - merged_df['SPREAD'])]
  

  merged_df = merged_df[['DATETIME', 'TICKER', 'SIZE', 'PRICE', 'BID', 'ASK']]
  merged_df.reset_index(drop=True, inplace=True)
  return merged_df

def get_all_taq():
  taq_df = pd.DataFrame()

  for year in range(2005, 2025):
    trade_file_path = f"./fomc_trades/trades_{year}.csv"
    quote_file_path = f"./fomc_quotes/quotes_{year}.csv"
    
    trades_df = pd.read_csv(trade_file_path)
    quotes_df = pd.read_csv(quote_file_path)

    curr_taq_df = get_taq(trades_df, quotes_df)
    taq_df = taq_df.append(curr_taq_df, ignore_index=True)
    print(f"completed {year}")

  return taq_df


file_path = f"./fomc_taq/taq_all.csv"

print(f"merging trade and quotes...")
df = get_all_taq()
print(f"outputing to csv at {file_path}")
df.to_csv(file_path, index=False)
print(f"complete!")
print(f"sample output:")
print(df.head())


merging trade and quotes...


/var/folders/yc/n6lh7krx6p3b0zmdngvdb48c0000gn/T/ipykernel_833/2996992556.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trades_df['DATETIME'] = pd.to_datetime(trades_df['DATE'] + ' ' + trades_df['TIME_M'], format='%Y-%m-%d %H:%M:%S.%f')
/var/folders/yc/n6lh7krx6p3b0zmdngvdb48c0000gn/T/ipykernel_833/2996992556.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quotes_df['DATETIME'] = pd.to_datetime(quotes_df['DATE'] + ' ' + quotes_df['TIME_M'], format='%Y-%m-%d %H:%M:%S.%f')
/var/folders/yc/n6lh7krx

completed 2005


/var/folders/yc/n6lh7krx6p3b0zmdngvdb48c0000gn/T/ipykernel_833/2996992556.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trades_df['DATETIME'] = pd.to_datetime(trades_df['DATE'] + ' ' + trades_df['TIME_M'], format='%Y-%m-%d %H:%M:%S.%f')
/var/folders/yc/n6lh7krx6p3b0zmdngvdb48c0000gn/T/ipykernel_833/2996992556.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quotes_df['DATETIME'] = pd.to_datetime(quotes_df['DATE'] + ' ' + quotes_df['TIME_M'], format='%Y-%m-%d %H:%M:%S.%f')
/var/folders/yc/n6lh7krx

completed 2006


...

completed 2022


/var/folders/yc/n6lh7krx6p3b0zmdngvdb48c0000gn/T/ipykernel_833/2996992556.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trades_df['DATETIME'] = pd.to_datetime(trades_df['DATE'] + ' ' + trades_df['TIME_M'], format='%Y-%m-%d %H:%M:%S.%f')
/var/folders/yc/n6lh7krx6p3b0zmdngvdb48c0000gn/T/ipykernel_833/2996992556.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quotes_df['DATETIME'] = pd.to_datetime(quotes_df['DATE'] + ' ' + quotes_df['TIME_M'], format='%Y-%m-%d %H:%M:%S.%f')
/var/folders/yc/n6lh7krx

completed 2023


/var/folders/yc/n6lh7krx6p3b0zmdngvdb48c0000gn/T/ipykernel_833/2996992556.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trades_df['DATETIME'] = pd.to_datetime(trades_df['DATE'] + ' ' + trades_df['TIME_M'], format='%Y-%m-%d %H:%M:%S.%f')
/var/folders/yc/n6lh7krx6p3b0zmdngvdb48c0000gn/T/ipykernel_833/2996992556.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quotes_df['DATETIME'] = pd.to_datetime(quotes_df['DATE'] + ' ' + quotes_df['TIME_M'], format='%Y-%m-%d %H:%M:%S.%f')
/var/folders/yc/n6lh7krx

completed 2024
outputing to csv at ./fomc_taq/taq_all.csv
complete!
sample output:
                 DATETIME TICKER   SIZE   PRICE     BID     ASK
0 2005-02-23 13:00:00.914    SPY  100.0  119.14  119.14  119.15
1 2005-02-23 13:00:00.917    SPY  100.0  119.14  119.14  119.15
2 2005-02-23 13:00:00.920    SPY  100.0  119.14  119.14  119.15
3 2005-02-23 13:00:00.922    SPY  100.0  119.14  119.14  119.15
4 2005-02-23 13:00:00.925    SPY  100.0  119.14  119.14  119.15
